In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from transformers import BertTokenizer, BertModel

class GraphConvolutionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvolutionLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, adj_matrix, features):
        support = torch.matmul(adj_matrix, features)
        output = self.linear(support)
        return output

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolutionLayer(input_dim, hidden_dim)
        self.gc2 = GraphConvolutionLayer(hidden_dim, output_dim)

    def forward(self, adj_matrix, features):
        h1 = F.relu(self.gc1(adj_matrix, features))
        output = self.gc2(adj_matrix, h1)
        return output

# Preprocess text and obtain BERT embeddings
def get_bert_embeddings(texts):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    encoded_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**encoded_texts)
    last_hidden_states = outputs.last_hidden_state
    return last_hidden_states

# Constructing adjacency matrix (assuming undirected graph for simplicity)
def construct_adjacency_matrix(features):
    num_nodes = len(features)
    adj_matrix = np.zeros((num_nodes, num_nodes))
    for i, feature_i in enumerate(features):
        for j, feature_j in enumerate(features):
            if i != j:
                # Compute cosine similarity between feature vectors
                similarity = cosine_similarity(features[feature_i], features[feature_j])
                adj_matrix[i, j] = similarity
    return torch.tensor(adj_matrix, dtype=torch.float)

# Example text data
texts = ["Example text 1", "Example text 2", "Example text 3"]

# Get BERT embeddings for text data
bert_embeddings = get_bert_embeddings(texts)

# Assuming input dimension is the sum of the BERT embedding size and the dimension of each feature vector
input_dim = bert_embeddings.shape[1] + len(features['emoji'])
hidden_dim = 16
output_dim = 1  # Output dimension can be adjusted based on your task

# Concatenate BERT embeddings with existing features
concatenated_features = torch.cat((bert_embeddings.mean(dim=1), torch.tensor(list(features.values()), dtype=torch.float)), dim=1)

# Construct adjacency matrix
adj_matrix = construct_adjacency_matrix(features)

# Initialize GCN model
gcn_model = GCN(input_dim, hidden_dim, output_dim)

# Forward pass
output = gcn_model(adj_matrix, concatenated_features)
print(output)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import numpy as np

class GraphConvolutionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GraphConvolutionLayer, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, adj_matrix, features):
        support = torch.matmul(adj_matrix, features)
        output = self.linear(support)
        return output

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolutionLayer(input_dim, hidden_dim)
        self.gc2 = GraphConvolutionLayer(hidden_dim, output_dim)

    def forward(self, adj_matrix, features):
        h1 = F.relu(self.gc1(adj_matrix, features))
        output = self.gc2(adj_matrix, h1)
        return output

# Constructing adjacency matrix (assuming undirected graph for simplicity)
def construct_adjacency_matrix(features):
    num_nodes = len(features)
    adj_matrix = np.zeros((num_nodes, num_nodes))
    for i, feature_i in enumerate(features):
        for j, feature_j in enumerate(features):
            if i != j:
                # Compute cosine similarity between feature vectors
                similarity = cosine_similarity(features[feature_i], features[feature_j])
                adj_matrix[i, j] = similarity
    return torch.tensor(adj_matrix, dtype=torch.float)

# Example usage:
features = {
    'emoji': [0.5, 0.3, 0.1],
    'hashtag': [0.2, 0.6, 0.4],
    'sentiment': [0.8, -0.2, 0.5],
    'semicolon': [1, 0, 0],
    'tweet_length': [0.7, 0.4, 0.9],
    'compound': [0.2, -0.5, 0.7],
    'neutral': [0.1, 0.9, -0.3],
    'url': [0.3, 0.2, 0.6],
    'cosine_similarity': [0.9, 0.8, 0.7]
}

# Assuming input dimension is the same as the dimension of each feature vector
input_dim = len(features['emoji'])
hidden_dim = 16
output_dim = 1  # Output dimension can be adjusted based on your task

# Construct adjacency matrix
adj_matrix = construct_adjacency_matrix(features)

# Initialize GCN model
gcn_model = GCN(input_dim, hidden_dim, output_dim)

# Forward pass
output = gcn_model(adj_matrix, torch.tensor(list(features.values()), dtype=torch.float))
print(output)


In [ ]:
import numpy as np

def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    similarity = dot_product / (magnitude1 * magnitude2)
    return similarity
